### Analyzing all-dates descriptives

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import datetime as dt
import time

from importlib import reload
from Utils import TransantiagoConstants
import TemporalDescriptivesBuilder

In [3]:
dates = TransantiagoConstants.updateCurrentSSHDates()
summary_dir = TransantiagoConstants.SummaryDir

* Only to re-factor

In [4]:
#date = dates[0]
dates = ['2017-03-15']

In [5]:
#start_index = dates.index('2017-08-22') + 1

In [6]:
#length_list = len(dates)

In [7]:
#sub_dates = dates[start_index:length_list]

In [8]:
#sub_dates

*****

* Building table (1) and (2) simultaneously

In [9]:
tic = time.clock()
general_summary = pd.DataFrame()
ind = 0

for date in dates:
    print(date + ' is being processed')
    temporal_descriptives_builder = TemporalDescriptivesBuilder.TemporalDescriptivesBuilderClass(date)
    
    ### General description of raw databases ###
    temporal_descriptives_builder.loadEtapasAndOthers()
    etapas_df = temporal_descriptives_builder.df
    total_trx = len(etapas_df.index)
    general_summary.loc[ind,'date'] = date
    general_summary.loc[ind,'total_trx'] = total_trx
    
    ### General description of clean databases ###
    temporal_descriptives_builder.processEtapas()
    etapas_df = temporal_descriptives_builder.df
    total_clean_trx = len(etapas_df.index)
    total_clean_turnstile_trx = len(etapas_df.loc[etapas_df['torniquete_mariposa']==1,:].index)
    total_clean_no_turnstile_trx = len(etapas_df.loc[etapas_df['no_torniquete']==1,:].index)
    
    avg_time_turnstile = etapas_df.loc[(etapas_df['torniquete_mariposa']==1) 
                                       &(~etapas_df['diferencia_tiempo_secs'].isnull()) 
                                       &(etapas_df['diferencia_tiempo_secs']<=etapas_df['diferencia_tiempo_secs'].quantile(.97)),'diferencia_tiempo_secs'].mean()
    
    avg_time_no_turnstile = etapas_df.loc[(etapas_df['no_torniquete']==1) 
                                       &(~etapas_df['diferencia_tiempo_secs'].isnull()) 
                                       &(etapas_df['diferencia_tiempo_secs']<=etapas_df['diferencia_tiempo_secs'].quantile(.97)),'diferencia_tiempo_secs'].mean()    
    
    std_time_turnstile = etapas_df.loc[(etapas_df['torniquete_mariposa']==1) 
                                       &(~etapas_df['diferencia_tiempo_secs'].isnull()) 
                                       &(etapas_df['diferencia_tiempo_secs']<=etapas_df['diferencia_tiempo_secs'].quantile(.97)),'diferencia_tiempo_secs'].std()
        
    std_time_no_turnstile = etapas_df.loc[(etapas_df['no_torniquete']==1) 
                                       &(~etapas_df['diferencia_tiempo_secs'].isnull()) 
                                       &(etapas_df['diferencia_tiempo_secs']<=etapas_df['diferencia_tiempo_secs'].quantile(.97)),'diferencia_tiempo_secs'].std()
    
    p_97 = etapas_df.loc[(~etapas_df['diferencia_tiempo_secs'].isnull())
                         &(etapas_df['diferencia_tiempo_secs']<=etapas_df['diferencia_tiempo_secs'].quantile(.97)),'diferencia_tiempo_secs'].max()
        
    
    general_summary.loc[ind,'total_clean_trx'] = total_clean_trx
    general_summary.loc[ind,'total_clean_turnstile_trx'] = total_clean_turnstile_trx
    general_summary.loc[ind,'total_clean_no_turnstile_trx'] = total_clean_no_turnstile_trx
    general_summary.loc[ind,'avg_time_turnstile'] = avg_time_turnstile
    general_summary.loc[ind,'avg_time_no_turnstile'] = avg_time_no_turnstile
    general_summary.loc[ind,'std_time_turnstile'] = std_time_turnstile
    general_summary.loc[ind,'std_time_no_turnstile'] = std_time_no_turnstile
    general_summary.loc[ind,'P97'] = p_97
    
    ind = ind+1
    ### Disaggregated analyses ###
    temporal_descriptives_builder.appendPeriods()
    temporal_descriptives_builder.groupData()
    temporal_descriptives_builder.appendUnidadNegocio()
    temporal_descriptives_builder.mergeTurnstileData()
    grouped_data = temporal_descriptives_builder.grouped_data    
    grouped_data['fecha']=date
    name_file = date + '_sumtrx.csv'
    disaggregated_path = os.path.join(summary_dir,name_file)
    grouped_data.to_csv(disaggregated_path,sep=';')

general_path = os.path.join(summary_dir,'summarytrx.csv')
general_summary.to_csv(general_path, sep=';')
toc = time.clock()
print(toc-tic)


2017-03-15 is being processed
59.98513152689055


In [10]:
general_summary.head()

,date,total_trx,total_clean_trx,total_clean_turnstile_trx,total_clean_no_turnstile_trx,avg_time_turnstile,avg_time_no_turnstile,std_time_turnstile,std_time_no_turnstile,P97
0,2017-03-15,2651580.0,2612985.0,167664.0,2039486.0,5.642612,3.436829,3.760756,3.545354,25.0
